# Portfolio Volatility

Measure realized and rolling volatility of the portfolio return stream.

**Data Source:**
- `wolfpack/daily_snapshots.csv` - portfolio NAV and daily returns

**Analysis:**
- Realized annualized volatility and downside volatility
- Rolling volatility (20/60/252-day)
- EWMA volatility estimate
- Volatility regime and monthly volatility diagnostics

**Prerequisites:** Run a WolfpackTrend backtest to populate `daily_snapshots.csv`.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from IPython.display import display

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

from QuantConnect import *
from QuantConnect.Research import QuantBook

qb = QuantBook()
print('QuantBook initialized')


def read_csv_from_store(key):
    try:
        if not qb.ObjectStore.ContainsKey(key):
            print(f'ObjectStore key not found: {key}')
            return None
        content = qb.ObjectStore.Read(key)
        if not content:
            print(f'Empty ObjectStore key: {key}')
            return None
        return pd.read_csv(StringIO(content))
    except Exception as e:
        print(f'Error reading {key}: {e}')
        return None

## Load Data and Compute Returns

In [ ]:
df_snapshots = read_csv_from_store('wolfpack/daily_snapshots.csv')

if df_snapshots is None:
    raise ValueError('daily_snapshots.csv is required. Run a backtest first.')

required_cols = ['date', 'nav']
missing = [c for c in required_cols if c not in df_snapshots.columns]
if missing:
    raise ValueError(f'daily_snapshots.csv missing required columns: {missing}')

df = df_snapshots[['date', 'nav']].copy()
df['date'] = pd.to_datetime(df['date'])
df['nav'] = pd.to_numeric(df['nav'], errors='coerce')

df = df.dropna(subset=['date', 'nav']).sort_values('date').reset_index(drop=True)
df['daily_return'] = df['nav'].pct_change()
df = df.dropna(subset=['daily_return']).copy()

print(f'Loaded {len(df)} return observations')
print(f'Date range: {df.date.min().strftime("%Y-%m-%d")} to {df.date.max().strftime("%Y-%m-%d")}')
display(df.tail())

## Helper Functions

In [ ]:
def annualized_volatility(returns, periods_per_year=252):
    return returns.std() * np.sqrt(periods_per_year)


def downside_volatility(returns, periods_per_year=252):
    downside = returns[returns < 0]
    if len(downside) == 0:
        return 0.0
    return downside.std() * np.sqrt(periods_per_year)


def ewma_volatility(returns, lam=0.94, periods_per_year=252):
    var = returns.ewm(alpha=1 - lam, adjust=False).var()
    return np.sqrt(var * periods_per_year)


print('Helper functions defined')


## Rolling Volatility Profile

In [ ]:
windows = [20, 60, 252]

for window in windows:
    df[f'vol_{window}'] = df['daily_return'].rolling(window).std() * np.sqrt(252)

df['ewma_vol'] = ewma_volatility(df['daily_return'])

fig, ax = plt.subplots(figsize=(14, 6))

for window in windows:
    col = f'vol_{window}'
    if df[col].notna().any():
        ax.plot(df['date'], df[col] * 100, linewidth=2, label=f'{window}-day')

ax.plot(df['date'], df['ewma_vol'] * 100, linewidth=2, color='black', alpha=0.8, label='EWMA (lambda=0.94)')
ax.axhline(10, color='red', linestyle='--', linewidth=1.8, label='Target Vol 10%')

ax.set_title('Rolling Annualized Volatility', fontsize=14, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Volatility (%)')
ax.grid(alpha=0.3)
ax.legend(loc='upper left')

plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## Distribution and Volatility Regime

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

sns.histplot(df['daily_return'] * 100, bins=50, kde=True, ax=axes[0], color='#1f77b4')
axes[0].axvline(0, color='black', linewidth=1)
axes[0].set_title('Daily Return Distribution', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Daily Return (%)')
axes[0].grid(alpha=0.3)

if df['vol_60'].notna().any():
    vol60 = df['vol_60'].dropna()
    p25 = vol60.quantile(0.25)
    p75 = vol60.quantile(0.75)

    latest_vol = vol60.iloc[-1]
    if latest_vol >= p75:
        regime = 'High-volatility regime'
    elif latest_vol <= p25:
        regime = 'Low-volatility regime'
    else:
        regime = 'Normal-volatility regime'

    axes[1].plot(df['date'], df['vol_60'] * 100, linewidth=2, color='darkorange', label='60-day volatility')
    axes[1].axhline(p25 * 100, color='green', linestyle='--', linewidth=1.4, label='25th percentile')
    axes[1].axhline(p75 * 100, color='red', linestyle='--', linewidth=1.4, label='75th percentile')
    axes[1].set_title(f'Volatility Regime: {regime}', fontsize=13, fontweight='bold')
    axes[1].set_xlabel('Date')
    axes[1].set_ylabel('Annualized Volatility (%)')
    axes[1].grid(alpha=0.3)
    axes[1].legend(loc='upper left')
else:
    axes[1].text(0.1, 0.5, 'Not enough data for 60-day volatility regime.', fontsize=11)
    axes[1].axis('off')

plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## Monthly Volatility Diagnostics

In [ ]:
df['year_month'] = df['date'].dt.to_period('M').astype(str)

monthly = (
    df.groupby('year_month', as_index=False)
      .agg(
          days=('daily_return', 'count'),
          monthly_return=('daily_return', lambda s: (1 + s).prod() - 1),
          monthly_vol=('daily_return', lambda s: s.std() * np.sqrt(252))
      )
)

monthly['monthly_return'] = monthly['monthly_return'] * 100
monthly['monthly_vol'] = monthly['monthly_vol'] * 100

display(monthly.tail(12))

plt.figure(figsize=(14, 5))
plt.plot(monthly['year_month'], monthly['monthly_vol'], marker='o', linewidth=2)
plt.title('Monthly Annualized Volatility', fontsize=13, fontweight='bold')
plt.xlabel('Month')
plt.ylabel('Volatility (%)')
plt.grid(alpha=0.3)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## Summary

In [ ]:
realized_vol = annualized_volatility(df['daily_return'])
downside_vol = downside_volatility(df['daily_return'])
max_drawdown = ((1 + df['daily_return']).cumprod() / (1 + df['daily_return']).cumprod().cummax() - 1).min()

summary_rows = [
    {'Metric': 'Realized annualized volatility', 'Value': realized_vol},
    {'Metric': 'Downside annualized volatility', 'Value': downside_vol},
    {'Metric': 'Max drawdown', 'Value': max_drawdown},
]

for window in windows:
    col = f'vol_{window}'
    val = df[col].dropna().iloc[-1] if df[col].notna().any() else np.nan
    summary_rows.append({'Metric': f'Latest {window}-day volatility', 'Value': val})

ewma_latest = df['ewma_vol'].dropna().iloc[-1] if df['ewma_vol'].notna().any() else np.nan
summary_rows.append({'Metric': 'Latest EWMA volatility', 'Value': ewma_latest})

summary = pd.DataFrame(summary_rows)
summary['Formatted'] = summary['Value'].apply(lambda x: f'{x * 100:.2f}%' if pd.notna(x) else 'n/a')

display(summary)

print('=' * 80)
print('VOLATILITY SUMMARY')
print('=' * 80)
print(f"Period: {df['date'].min().strftime('%Y-%m-%d')} to {df['date'].max().strftime('%Y-%m-%d')}")
print(f'Observations: {len(df)}')
print(f'Realized annualized volatility: {realized_vol * 100:.2f}%')
print(f'Downside annualized volatility: {downside_vol * 100:.2f}%')
print(f'Max drawdown: {max_drawdown * 100:.2f}%')
print('=' * 80)